In [10]:
#importing necessary libraries
import glob
import nltk
import os
import pandas as pd
import numpy as np
import json
import gensim
import tensorflow as tf

#gensim libraries
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import preprocess_documents
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from gensim.parsing.preprocessing import strip_tags, remove_stopwords, STOPWORDS
from sklearn.metrics import f1_score
from gensim.models import Word2Vec

#sk-learn libraries
from sklearn.cluster import KMeans

#confg libraries
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:
df = pd.read_excel("abstract_excel.xlsx") # reading excel file/csv file.

In [4]:
df.head(4)

,Title,Abstract,Filename
0,One-stop shops for residential building energy renovation in the EU,"This report is the second part of a Europe-wide survey and assessment of one-stop shops (OSS) for energy renovation of buildings. Based on the case studies of 63 OSS in the EU, we find that the approach has a potential to cover 5-6% of the renovation volume of 35 million buildings in 2030 set out by the Renovation Wave Strategy1,2 at low social costs, integrating private investments with client-friendly methods. The current level of activity of the European OSS market is estimated to be around 100 000 projects per year.\nThe OSS can bridge the gap between a fragmented residential building sector, with a large heterogeneous set of households, and the construction supply side. They can help increase the actual renovation rate by supporting potential clients through the various steps of the decision-making process. Their success lies in part with their locally embedded focus, engagement with interested but not yet committed energy users/asset owners and ability to form strong relationships with clients. They can support the renovation journey from start to finish and can facilitate access to financing, occasionally offering better rates. While our analysis identifies several structural, legislative, financial, and information measures with a positive impact on OSS-enabled renovation projects, it is possible to enhance their contribution by reviewing specific policies that support or hinder their success. Although not typical of OSS in general, but with proper incentives, they can improve the average renovation depth in terms of energy performance level by adopting a holistic approach and, at the same time, they can reach out to vulnerable populations, such as tenants of social houses, thus contributing to the alleviation of energy poverty. Finally, OSS can contribute to the enhancement of communities and neighbourhoods. They help current tenants to improve their living conditions and thus stay in the area.",One-stop shops for residential building.pdf
1,HOW BUILDING INSULATION CAN REDUCE FOSSIL FUEL IMPORTS AND BOOST EU ENERGY SECURITY,"This paper investigates how improving the energy performance of the building envelope in selected EU Member States would lower fossil fuel consumption, increase energy security and enable the growth of renewable heat. The study shows how improving insulation can reduce energy demand and drastically cut Russian oil and gas imports for heating in buildings. Investing in building renovation can reduce the use of fossil fuels for heating in buildings, reaching 44% in gas savings, save 45% of final energy demand and substantially contribute to securing the EU’s energy needs.",HOW BUILDING INSULATION CAN REDUCE.pdf
2,Impact assessment of the MEPS under discussion in the context of the EPBD,"Minimum Energy Performance Standards (MEPS) are under discussion in the frame of the revision of the Energy Performance of Buildings Directive (EPBD) as a tool to trigger renovation of buildings in the European Union and reduce GHG emissions. To inform this discussion, Climact and BPIE modelled the impacts of MEPS schemes on the European building stock. This allows comparing the ambition of MEPS as originally proposed by the European Commission with two other scenarios: the REPowerEU plan and a scenario aligned with the EU climate and energy consumption reduction targets set by Fitfor552. The modelling was done in June 2022 and mainly accounts for the proposals from the EU Commission, REPowerEU and the EP rapporteur for the EPBD revision, Ciáran Cuffe.",Impact assessment of the MEPS.pdf
3,Recommendations for industrialised renovation,"Action and innovation are needed to achieve the EU climate goal of full, societywide\ndecarbonisation by 2050. The EU is committed to a 55% reduction of\ngreenhouse-gas emissions by 2030.1 The 2022 REPowerEU plan puts increased\nemphasis on decarbonisation, notably through reducing energy dema

In [5]:
## creating a list for storing the abstract and filename column 
abstract_list = df['Abstract']
filename_list = df["Filename"]

In [6]:
#checking
abstract_list = df['Abstract']

#### Preprocessing of the text  

1. remove punctuation and stopwords.
2. implement lemmatization
3. unnecessary white space chr and space. +++

In [7]:
#stop word removal 
import spacy
nlp=spacy.load('en_core_web_sm')
import string
from spacy.lang.en.stop_words import STOP_WORDS

In [8]:
punctuations = string.punctuation
stop_words = list(STOP_WORDS)

custom_stop_words = [
    'doi', 'preprint', 'copyright', 'peer', 'reviewed', 'org', 'https', 'et', 'al', 'author', 'figure', 
    'rights', 'reserved', 'permission', 'used', 'using', 'license', 'fig', 'fig.', 
    'al.', 'Elsevier'
]

for w in custom_stop_words:
    if w not in stop_words:
        stop_words.append(w)

In [9]:
def tokenize_corpus(abstract_text):
    mytokens = nlp(abstract_text)
    mytokens = [ word.lemma_.strip() if word.lemma_ != "-PRON-" else word for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

In [25]:
df["processed_text"] = df["Abstract"].apply(tokenize_corpus)

In [27]:
df.head(3)

,Title,Abstract,Filename,processed_text
0,One-stop shops for residential building energy renovation in the EU,"This report is the second part of a Europe-wide survey and assessment of one-stop shops (OSS) for energy renovation of buildings. Based on the case studies of 63 OSS in the EU, we find that the approach has a potential to cover 5-6% of the renovation volume of 35 million buildings in 2030 set out by the Renovation Wave Strategy1,2 at low social costs, integrating private investments with client-friendly methods. The current level of activity of the European OSS market is estimated to be around 100 000 projects per year.\nThe OSS can bridge the gap between a fragmented residential building sector, with a large heterogeneous set of households, and the construction supply side. They can help increase the actual renovation rate by supporting potential clients through the various steps of the decision-making process. Their success lies in part with their locally embedded focus, engagement with interested but not yet committed energy users/asset owners and ability to form strong relationships with clients. They can support the renovation journey from start to finish and can facilitate access to financing, occasionally offering better rates. While our analysis identifies several structural, legislative, financial, and information measures with a positive impact on OSS-enabled renovation projects, it is possible to enhance their contribution by reviewing specific policies that support or hinder their success. Although not typical of OSS in general, but with proper incentives, they can improve the average renovation depth in terms of energy performance level by adopting a holistic approach and, at the same time, they can reach out to vulnerable populations, such as tenants of social houses, thus contributing to the alleviation of energy poverty. Finally, OSS can contribute to the enhancement of communities and neighbourhoods. They help current tenants to improve their living conditions and thus stay in the area.",One-stop shops for residential building.pdf,"report second Europe wide survey assessment stop shop OSS energy renovation building base case study 63 OSS EU find approach potential cover 5 6 renovation volume 35 million building 2030 set Renovation Wave Strategy1,2 low social cost integrate private investment client friendly method current level activity European OSS market estimate 100 000 project year OSS bridge gap fragmented residential building sector large heterogeneous set household construction supply help increase actual renovation rate support potential client step decision process success lie locally embed focus engagement interested commit energy user asset owner ability form strong relationship client support renovation journey start finish facilitate access financing occasionally offer rate analysis identify structural legislative financial information measure positive impact oss enable renovation project possible enhance contribution review specific policy support hinder success typical OSS general proper incentive improve average renovation depth term energy performance level adopt holistic approach time reach vulnerable population tenant social house contribute alleviation energy poverty finally OSS contribute enhancement community neighbourhood help current tenant improve living condition stay area"
1,HOW BUILDING INSULATION CAN REDUCE FOSSIL FUEL IMPORTS AND BOOST EU ENERGY SECURITY,"This paper investigates how improving the energy performance of the building envelope in selected EU Member States would lower fossil fuel consumption, increase energy security and enable the growth of renewable heat. The study shows how improving insulation can reduce energy demand and drastically cut Russian oil and gas imports for heating in buildings. Investing in building renovation can reduce the use of fossil fuels for heating in buildings, reaching 44% in gas savings, save 45% of final energy demand and substanti

#### Applying Word-Embedding

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
def vectorize(text, maxx_features):
    
    vectorizer = TfidfVectorizer(max_features=maxx_features)
    X = vectorizer.fit_transform(text)
    #print(vectorizer.get_feature_names_out())
    return X

In [29]:
pro_text = df['processed_text'].values
max_features = 2**8

X = vectorize(pro_text, max_features)

In [31]:
print(X)

  (0, 15)	0.08877182185389655
  (0, 39)	0.08759075603194663
  (0, 44)	0.18084051002366922
  (0, 101)	0.10824653290300298
  (0, 183)	0.0959695919226806
  (0, 239)	0.07918778832120299
  (0, 163)	0.054690348929732624
  (0, 236)	0.08228044665206914
  (0, 19)	0.10402624043691523
  (0, 108)	0.15663457171321968
  (0, 167)	0.07306727072796335
  (0, 219)	0.09861229498241474
  (0, 201)	0.08395116671609551
  (0, 168)	0.09171698208155171
  (0, 72)	0.0954671289176706
  (0, 104)	0.06005898529407443
  (0, 135)	0.07506156227159791
  (0, 116)	0.0935352002487891
  (0, 103)	0.07610914713457896
  (0, 10)	0.06636569384699177
  (0, 247)	0.09916908110827477
  (0, 87)	0.08038797000763628
  (0, 173)	0.0813197228271735
  (0, 52)	0.08682773469815665
  (0, 228)	0.23409745168885965
  :	:
  (499, 216)	0.1612023290246324
  (499, 211)	0.08838353016870806
  (499, 96)	0.10496249461223313
  (499, 2)	0.07170811875664732
  (499, 232)	0.08975641405952066
  (499, 71)	0.13491158887077512
  (499, 246)	0.04744583192746147
  (4

#### Application of clustering algorithm

In [ ]:
from sklearn import metrics
from scipy.spatial.distance import cdist

# run kmeans with many different k
distortions = []
K = range(2, 20)
for k in K:
    k_means = KMeans(n_clusters=k, random_state=42).fit(X)
    k_means.fit(X)
    distortions.append(sum(np.min(cdist(X.toarray(), k_means.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])

In [11]:
## perform elbow method
## decide on number of cluster based on elbow method 
## try to visulaize in the graph 
## vectorize the data 
## perform topic modeling
## append the keywords to df